In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, Conv1D, GlobalMaxPooling1D, MultiHeadAttention, LayerNormalization, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback
import tensorflow as tf

# Data Preprocessing
def preprocess_data(df):
    event_type_mapping = df['event_type'].astype('category').cat.categories
    agent_id_mapping = df['agent_id'].astype('category').cat.categories
    context_mapping = df['context'].astype('category').cat.categories

    df['event_type'] = df['event_type'].astype('category').cat.codes
    df['agent_id'] = df['agent_id'].astype('category').cat.codes
    df['context'] = df['context'].astype('category').cat.codes

    return df, event_type_mapping, agent_id_mapping, context_mapping

# Load your dataset
df = pd.read_csv('1k_single_agent_minmax.csv')
df, event_type_mapping, agent_id_mapping, context_mapping = preprocess_data(df)

# Define constants
num_classes = df['event_type'].nunique()
num_agents = df['agent_id'].nunique()
num_contexts = df['context'].nunique()
sequence_length = 10  # Adjust based on your data
num_features = df.shape[1]  # Number of features

# Create sequences for training
def create_sequences(df, sequence_length):
    sequences = []
    for i in range(len(df) - sequence_length):
        seq = df.iloc[i:i+sequence_length].values
        sequences.append(seq)
    return np.array(sequences)

sequences = create_sequences(df, sequence_length)

# Splitting the data
train_size = int(0.7 * len(sequences))
val_size = int(0.15 * len(sequences))
train_sequences = sequences[:train_size]
val_sequences = sequences[train_size:train_size+val_size]
test_sequences = sequences[train_size+val_size:]

# Extract targets
def extract_targets(sequences):
    next_event = to_categorical(sequences[:, -1, 0], num_classes=num_classes)
    agent = to_categorical(sequences[:, -1, 1], num_classes=num_agents)
    context = to_categorical(sequences[:, -1, 2], num_classes=num_contexts)
    anomaly = np.random.randint(0, 2, size=(sequences.shape[0], 1))  # Placeholder for anomaly
    return next_event, agent, context, anomaly

train_targets = extract_targets(train_sequences)
val_targets = extract_targets(val_sequences)
test_targets = extract_targets(test_sequences)

# Define the model branches
def lstm_branch(input_shape):
    inputs = Input(shape=(input_shape[0],))
    x = Embedding(input_dim=num_classes, output_dim=64)(inputs)
    x = LSTM(128, return_sequences=False)(x)
    return inputs, x

def transformer_branch(input_shape):
    inputs = Input(shape=(input_shape[0],))
    x = Embedding(input_dim=num_classes, output_dim=64)(inputs)
    attn_output = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
    x = LayerNormalization()(x + attn_output)
    x = GlobalMaxPooling1D()(x)  # Apply GlobalMaxPooling1D to convert 3D to 2D
    return inputs, x

def convnet_branch(input_shape):
    inputs = Input(shape=(input_shape[0],))
    x = Embedding(input_dim=num_classes, output_dim=64)(inputs)
    x = Conv1D(128, kernel_size=3, activation='relu')(x)
    x = GlobalMaxPooling1D()(x)
    return inputs, x

# Hierarchical Attention
def hierarchical_attention_branch(input_shape):
    inputs = Input(shape=(input_shape[0], input_shape[1]))
    x = Attention()([inputs, inputs])
    x = GlobalMaxPooling1D()(x)
    return inputs, x

# Build the model
def build_model(sequence_length):
    # Define the branches for each input
    lstm_event_inputs, lstm_event_output = lstm_branch((sequence_length,))
    lstm_agent_inputs, lstm_agent_output = lstm_branch((sequence_length,))
    lstm_context_inputs, lstm_context_output = lstm_branch((sequence_length,))
    
    transformer_event_inputs, transformer_event_output = transformer_branch((sequence_length,))
    transformer_agent_inputs, transformer_agent_output = transformer_branch((sequence_length,))
    transformer_context_inputs, transformer_context_output = transformer_branch((sequence_length,))
    
    convnet_event_inputs, convnet_event_output = convnet_branch((sequence_length,))
    convnet_agent_inputs, convnet_agent_output = convnet_branch((sequence_length,))
    convnet_context_inputs, convnet_context_output = convnet_branch((sequence_length,))
    
    # Hierarchical Attention
    hier_attn_inputs, hier_attn_output = hierarchical_attention_branch((sequence_length, num_features))
    
    # Concatenate outputs from all branches
    concatenated = Concatenate()([
        lstm_event_output, lstm_agent_output, lstm_context_output,
        transformer_event_output, transformer_agent_output, transformer_context_output,
        convnet_event_output, convnet_agent_output, convnet_context_output,
        hier_attn_output
    ])
    
    shared_dense = Dense(128, activation='relu')(concatenated)
    
    next_event_head = Dense(num_classes, activation='softmax', name='next_event')(shared_dense)
    agent_head = Dense(num_agents, activation='softmax', name='agent')(shared_dense)
    context_head = Dense(num_contexts, activation='softmax', name='context')(shared_dense)
    anomaly_head = Dense(1, activation='sigmoid', name='anomaly')(shared_dense)
    
    model = Model(inputs=[
        lstm_event_inputs, lstm_agent_inputs, lstm_context_inputs,
        transformer_event_inputs, transformer_agent_inputs, transformer_context_inputs,
        convnet_event_inputs, convnet_agent_inputs, convnet_context_inputs,
        hier_attn_inputs
    ], outputs=[next_event_head, agent_head, context_head, anomaly_head])
    
    return model

input_shape = (sequence_length,)
model = build_model(sequence_length)

# Dynamic loss weighting using uncertainty weighting
class CustomLossWeights(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        loss_weights = {
            'next_event': 1.0,
            'agent': 0.5,
            'context': 0.5,
            'anomaly': 0.1
        }
        # Update loss weights dynamically based on logs if necessary
        for key in loss_weights.keys():
            if f'{key}_loss' in logs:
                loss_weights[key] = 1.0 / (logs[f'{key}_loss'] + 1e-5)
        
        self.model.compile(optimizer=self.model.optimizer, loss=self.model.loss, loss_weights=loss_weights)

# Meta-learning inspired custom training loop
class MetaLearning(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        optimizer = self.model.optimizer
        # Ensure learning rate is a TensorFlow variable
        if isinstance(optimizer.learning_rate, tf.Variable):
            lr = tf.keras.backend.get_value(optimizer.learning_rate)
            new_lr = lr * (1 / (1 + 0.1 * epoch))
            tf.keras.backend.set_value(optimizer.learning_rate, new_lr)
        else:
            print("Optimizer's learning rate is not a TensorFlow variable. Update the learning rate handling accordingly.")

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss={
        'next_event': 'categorical_crossentropy', 
        'agent': 'categorical_crossentropy',
        'context': 'categorical_crossentropy', 
        'anomaly': 'binary_crossentropy'
    },
    loss_weights={
        'next_event': 1.0, 
        'agent': 0.5, 
        'context': 0.5, 
        'anomaly': 0.1
    },
    metrics={
        'next_event': 'accuracy', 
        'agent': 'accuracy',
        'context': 'accuracy',
        'anomaly': 'accuracy'
    }
)

# Training the model
history = model.fit(
    [train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2],
     train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2],
     train_sequences[:,:,0], train_sequences[:,:,1], train_sequences[:,:,2],
     train_sequences],
    {'next_event': train_targets[0], 'agent': train_targets[1], 'context': train_targets[2], 'anomaly': train_targets[3]},
    validation_data=(
        [val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
         val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
         val_sequences[:,:,0], val_sequences[:,:,1], val_sequences[:,:,2],
         val_sequences],
        {'next_event': val_targets[0], 'agent': val_targets[1], 'context': val_targets[2], 'anomaly': val_targets[3]}
    ),
    epochs=10, batch_size=32,
    callbacks=[CustomLossWeights(), MetaLearning()]
)

# Evaluate the model
eval_results = model.evaluate(
    [test_sequences[:,:,0], test_sequences[:,:,1], test_sequences[:,:,2],
     test_sequences[:,:,0], test_sequences[:,:,1], test_sequences[:,:,2],
     test_sequences[:,:,0], test_sequences[:,:,1], test_sequences[:,:,2],
     test_sequences],
    {'next_event': test_targets[0], 'agent': test_targets[1], 'context': test_targets[2], 'anomaly': test_targets[3]}
)
print(f"Test Loss and Accuracy: {eval_results}")


Epoch 1/10
20/21 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - agent_accuracy: 0.4200 - anomaly_accuracy: 0.4391 - context_accuracy: 0.0900 - loss: 2.8277 - next_event_accuracy: 0.6659Optimizer's learning rate is not a TensorFlow variable. Update the learning rate handling accordingly.
21/21 ━━━━━━━━━━━━━━━━━━━━ 25s 142ms/step - agent_accuracy: 0.4232 - anomaly_accuracy: 0.4433 - context_accuracy: 0.0892 - loss: 2.8095 - next_event_accuracy: 0.6738 - val_agent_accuracy: 0.7113 - val_anomaly_accuracy: 0.5141 - val_context_accuracy: 0.0986 - val_loss: 2.3907 - val_next_event_accuracy: 0.8028
Epoch 2/10


TypeError: 'NoneType' object is not callable

In [2]:
num_samples = 10
sample_sequences = test_sequences[:num_samples]

# Predict the next values
predictions = model.predict(
    [sample_sequences[:,:,0], sample_sequences[:,:,1], sample_sequences[:,:,2],
     sample_sequences[:,:,0], sample_sequences[:,:,1], sample_sequences[:,:,2],
     sample_sequences[:,:,0], sample_sequences[:,:,1], sample_sequences[:,:,2]]
)

# Extract predictions
next_event_predictions = np.argmax(predictions[0], axis=-1)
agent_predictions = np.argmax(predictions[1], axis=-1)
context_predictions = np.argmax(predictions[2], axis=-1)
anomaly_predictions = (predictions[3] > 0.5).astype(int)

# Decode predictions
def decode_predictions(predictions, id_to_label):
    return [id_to_label[idx] for idx in predictions]

decoded_next_event_predictions = decode_predictions(next_event_predictions, event_type_mapping)
decoded_agent_predictions = decode_predictions(agent_predictions, agent_id_mapping)
decoded_context_predictions = decode_predictions(context_predictions, context_mapping)

# Decode input sequences
def decode_sequences(sequences, event_mapping, agent_mapping, context_mapping):
    decoded_sequences = []
    for seq in sequences:
        decoded_seq = []
        for step in seq:
            decoded_step = [
                event_mapping[step[0]],
                agent_mapping[step[1]],
                context_mapping[step[2]]
            ]
            decoded_seq.append(decoded_step)
        decoded_sequences.append(decoded_seq)
    return np.array(decoded_sequences)

decoded_sample_sequences = decode_sequences(sample_sequences, event_type_mapping, agent_id_mapping, context_mapping)

# Print decoded input sequences and predictions
for i in range(num_samples):
    print(f"Input sequence (event_id, agent_id, context):\n{decoded_sample_sequences[i]}")
    print(f"Predicted next event_id: {decoded_next_event_predictions[i]}")
    print(f"Predicted next agent_id: {decoded_agent_predictions[i]}")
    print(f"Predicted next context: {decoded_context_predictions[i]}")
    print(f"Predicted anomaly: {anomaly_predictions[i]}")
    print("-" * 30)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 848ms/step
Input sequence (event_id, agent_id, context):
[['GAME_START' 'system' 'New Game']
 ['MOVE' 'X' '1,0']
 ['MOVE' 'O' '0,2']
 ['MOVE' 'X' '2,1']
 ['MOVE' 'O' '2,2']
 ['MOVE' 'X' '1,2']
 ['MOVE' 'O' '1,1']
 ['MOVE' 'X' '0,0']
 ['MOVE' 'O' '2,0']
 ['GAME_END' 'system' 'O']]
Predicted next event_id: GAME_END
Predicted next agent_id: system
Predicted next context: O
Predicted anomaly: [1]
------------------------------
Input sequence (event_id, agent_id, context):
[['MOVE' 'X' '1,0']
 ['MOVE' 'O' '0,2']
 ['MOVE' 'X' '2,1']
 ['MOVE' 'O' '2,2']
 ['MOVE' 'X' '1,2']
 ['MOVE' 'O' '1,1']
 ['MOVE' 'X' '0,0']
 ['MOVE' 'O' '2,0']
 ['GAME_END' 'system' 'O']
 ['GAME_START' 'system' 'New Game']]
Predicted next event_id: GAME_START
Predicted next agent_id: system
Predicted next context: New Game
Predicted anomaly: [1]
------------------------------
Input sequence (event_id, agent_id, context):
[['MOVE' 'O' '0,2']
 ['MOVE' 'X' '2,1']
 ['MOVE' 'O' '2,2']
 ['MOVE' 'X' '